<a href="https://colab.research.google.com/github/jamesETsmith/2022_simons_collab_pyscf_workshop/blob/main/demos/00_PySCF_Intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up the Jupyter notebook

* We need to install a few things before we get started
  * [PySCF](https://pyscf.org/) (for the quantum chemistry)
  * [py3DMol](https://3dmol.csb.pitt.edu/) for visualizing the molecule
  * [plotly](https://plotly.com/python/) and kaleido for plotting

In [ ]:
%pip install -q pyscf py3DMol plotly kaleido

# Loading the PySCF Package

In [ ]:
from pyscf import gto   # Used to define a molecule
from pyscf import scf   # Used to perform HF calculations
from pyscf import mp    # Used to perform Møller–Plesset PT calculations
from pyscf import cc    # Used to perfrom Coupled Cluster calculations
from pyscf import mcscf # Used to perform multireference calculations

# Setting Up Our System

In [ ]:
mol_xyz = """H      1.2194     -0.1652      2.1600
  C      0.6825     -0.0924      1.2087
  C     -0.7075     -0.0352      1.1973
  H     -1.2644     -0.0630      2.1393
  C     -1.3898      0.0572     -0.0114
  H     -2.4836      0.1021     -0.0204
  C     -0.6824      0.0925     -1.2088
  H     -1.2194      0.1652     -2.1599
  C      0.7075      0.0352     -1.1973
  H      1.2641      0.0628     -2.1395
  C      1.3899     -0.0572      0.0114
  H      2.4836     -0.1022      0.0205"""

mol = gto.M(atom=mol_xyz, basis="sto3g", verbose=3)

# Visualizing Our System

Working in Python (and Jupyter notebooks) has a lot of advantages. One of those is being able to visualize our system with other Python packages such as `py3Dmol`.

In [ ]:
import py3Dmol

xyz_view = py3Dmol.view(width=400,height=400)
xyz_view.addModel(mol.tostring(format="xyz"),'xyz')
xyz_view.setStyle({'stick':{}, "sphere":{"radius":0.4}})
xyz_view.setBackgroundColor('0xeeeeee')
xyz_view.show()

#
# Use your mouse to interact with the molecule!
#

# Running Simple Calculations

In [ ]:
# Run HF and save the results
mf = scf.RHF(mol).run()

# In Jupyter notebooks you can hover over methods to see docstrings or you can print the docstrings out!
print(mf.__doc__)

# Visualizing Results

Even from "simple" mean field methods, there are many important results and working in Python makes it easy to manipulate and visualize them!

In [ ]:
import plotly.express as px

# Plot the MO Occupations
fig = px.line(y=mf.mo_occ, markers=True, title="Molecular Orbital (MO) Occupations")
fig.update_layout(xaxis_title="Orbital Index (0-Based)", yaxis_title="MO Occupation")
fig.show()

In [ ]:
# Plot the MO Energies (i.e. eigenvalues of the Fock matrix)
fig = px.line(y=mf.mo_energy, markers=True, title="Molecular Orbital (MO) Energies")
fig.update_layout(xaxis_title="Orbital Index (0-Based)", yaxis_title="MO Energies")
fig.show()

In [ ]:
from pyscf.tools import cubegen

cubegen.orbital(mol, 'mo.cube', mf.mo_coeff[:,20]);

cube_view = py3Dmol.view(width=400,height=400)
cube_view.addVolumetricData(open("mo.cube").read(), "cube", {'isoval': -0.03, 'color': "red", 'opacity': 0.85})
cube_view.addVolumetricData(open("mo.cube").read(), "cube", {'isoval': 0.03, 'color': "blue", 'opacity': 0.85})
cube_view.addModel(mol.tostring(format="xyz"),'xyz')
cube_view.setStyle({'stick':{}, "sphere":{"radius":0.4}})
cube_view.setBackgroundColor('0xeeeeee')
cube_view.show()

#
# Use your mouse to interact with the molecule!
#

# Working with Model Hamiltonians

It's easy to customize the Hamiltonian for work with models. The one- and two-body interactions are just `numpy.ndarray` objects.

In the example below, we show how to set up a 1D anti-PBD Hubbard model and perform meanfield *and* correlated calculations.

In [ ]:
# 1D anti-PBC Hubbard model at half filling
# See https://github.com/pyscf/pyscf/blob/master/examples/mcscf/40-customizing_hamiltonian.py

import numpy as np
from pyscf import ao2mo

mol = gto.M(verbose=3)
mol.nelectron = 6

mol.incore_anyway = True


n = 12

h1 = np.zeros((n,n))
for i in range(n-1):
    h1[i,i+1] = h1[i+1,i] = -1.0
h1[n-1,0] = h1[0,n-1] = -1.0
eri = np.zeros((n,n,n,n))
for i in range(n):
    eri[i,i,i,i] = 2.0

mf = scf.RHF(mol)
mf.get_hcore = lambda *args: h1
mf.get_ovlp = lambda *args: np.eye(n)
mf._eri = ao2mo.restore(8, eri, n)
mf.init_guess = '1e'
mf.kernel()

mycc = cc.CCSD(mf).run()

mycas = mcscf.CASSCF(mf, 4, 4)
mycas.kernel();

# Working with Periodic Systems

In PySCF, molecules and periodic systems are treated (intentionally) in a very similar way!

In [ ]:
from pyscf.pbc import gto as pbcgto
from pyscf.pbc import scf as pbcscf

cell = pbcgto.M(
    atom = '''C 0.0000 0.0000 0.0000
              C 0.8917 0.8917 0.8917''',
    a = '''0.0000 1.7834 1.7834
           1.7834 0.0000 1.7834
           1.7834 1.7834 0.0000''',
    pseudo = 'gth-pade',
    basis = 'gth-szv'
    )
kpts = [2,2,2]
mf = pbcscf.KRKS(cell, cell.make_kpts(kpts), xc="lda").run()